In [103]:
import itertools
from itertools import combinations
from pathlib import Path

import krippendorff
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.metrics import classification_report, cohen_kappa_score

In [21]:
dataset_root = Path(open("../../dataset_root.txt").read().strip())

# Human Judges

In [22]:
df_annotations = pd.read_csv(dataset_root / "Human Panel Samples" / "annotations.csv")
df_annotators = pd.read_csv(dataset_root / "Human Panel Samples" / "annotators.csv")

In [78]:
def merge_annotation_data(df_annotations, df_annotators):
    sessions = df_annotations.submission_id.to_list()
    engagement_cols = [f"session#{s}" for s in sessions]
    confidence_cols = [f"session#{s}_confidence" for s in sessions]

    annotations = df_annotations.set_index("submission_id")
    df = df_annotators.copy()

    for session in sessions:
        df[f"session#{session}"] = [
            annotations.loc[session][f"annotator_{i}"] for i in range(14)
        ]
        df[f"session#{session}_confidence"] = [
            annotations.loc[session][f"annotator_{i}_conf"] for i in range(14)
        ]
    return df

In [82]:
df = merge_annotation_data(df_annotations, df_annotators)
df.head()

,annotator_id,experience,clues,session#201,session#201_confidence,session#207,session#207_confidence,session#212,session#212_confidence,session#224,...,session#869,session#869_confidence,session#870,session#870_confidence,session#872,session#872_confidence,session#880,session#880_confidence,session#885,session#885_confidence
0,0,4,"Blinking, Head position, and Eye movement",2,2,2,2,0,2,2,...,2,0,4,2,0,2,2,4,4,2
1,1,4,Eye blinking Mouth movements Eye movement betw...,2,3,3,3,2,1,0,...,3,2,4,3,2,1,3,2,0,2
2,2,2,"Eye movement, Facial expressions like mouth mo...",3,3,3,3,3,4,3,...,2,1,3,0,4,3,2,2,2,2
3,3,2,"their eye movements, their focus on the screen...",2,3,2,3,3,3,3,...,1,2,1,3,1,3,2,3,2,3
4,4,4,"Eye and head movements, blinking rate, general...",3,3,4,4,4,3,4,...,4,3,3,3,2,4,4,4,4,4


In [83]:
engagement_cols = [
    col for col in df.columns if "session" in col and "confidence" not in col
]
confidence_cols = [col for col in df.columns if "confidence" in col]

In [89]:
def analyze_agreement(df):
    # Extract just the engagement ratings (exclude confidence scores)
    ratings = df[engagement_cols].values

    # Krippendorff's alpha
    k_alpha = alpha(reliability_data=ratings)

    # Pairwise Cohen's Kappa
    judge_pairs = list(itertools.combinations(range(len(df)), 2))
    kappas = []
    for i, j in judge_pairs:
        kappa = cohen_kappa_score(ratings[i], ratings[j])
        kappas.append(kappa)

    avg_kappa = np.mean(kappas)

    # Raw agreement percentage
    raw_agreement = np.mean([np.mean(ratings[i] == ratings[j]) for i, j in judge_pairs])

    return {
        "krippendorff_alpha": k_alpha,
        "average_cohens_kappa": avg_kappa,
        "raw_agreement": raw_agreement,
    }

In [90]:
analyze_agreement(df)

{'krippendorff_alpha': 0.09186384578796136,
 'average_cohens_kappa': 0.0009328166107433642,
 'raw_agreement': 0.2258241758241758}

In [91]:
def analyze_agreement_with_confidence(df):
    # Get engagement and confidence columns
    ratings = df[engagement_cols].values
    confidences = df[confidence_cols].values

    # Weighted agreement based on confidence
    judge_pairs = list(itertools.combinations(range(len(df)), 2))
    weighted_agreements = []

    for i, j in judge_pairs:
        # Normalize confidence scores to 0-1
        conf_weight = (confidences[i] + confidences[j]) / (2 * confidences.max())
        agreement = ratings[i] == ratings[j]
        weighted_agreement = np.mean(agreement * conf_weight)
        weighted_agreements.append(weighted_agreement)

    return {
        "weighted_agreement": np.mean(weighted_agreements),
        "high_confidence_agreement": np.mean(
            [wa for wa, w in zip(weighted_agreements, conf_weight) if w > 0.8]
        ),
    }

In [92]:
analyze_agreement_with_confidence(df)

{'weighted_agreement': 0.15295329670329672,
 'high_confidence_agreement': 0.09687499999999999}

### Aggregating labels

In [93]:
agg_df = df.copy()
agg_df[engagement_cols] = agg_df[engagement_cols].replace(
    {0: 0, 1: 0, 2: 1, 3: 3, 4: 3}
)
analyze_agreement(agg_df)

{'krippendorff_alpha': 0.07972979758482346,
 'average_cohens_kappa': 0.036495584029178525,
 'raw_agreement': 0.378021978021978}

In [94]:
analyze_agreement_with_confidence(agg_df)

{'weighted_agreement': 0.26696428571428577,
 'high_confidence_agreement': 0.23124999999999998}

### Binarizing labels

In [95]:
bin_df = df.copy()
bin_df[engagement_cols] = bin_df[engagement_cols].replace(
    {0: 0, 1: 0, 2: 0, 3: 3, 4: 3}
)
analyze_agreement(bin_df)

{'krippendorff_alpha': 0.04035114738949608,
 'average_cohens_kappa': 0.04342727663514044,
 'raw_agreement': 0.5214285714285714}

In [96]:
analyze_agreement_with_confidence(bin_df)

{'weighted_agreement': 0.36064560439560434,
 'high_confidence_agreement': 0.340625}

### Human Performance

In [97]:
df_annotations.head()

,participant_id,submission_id,engagement,annotator_0,annotator_0_conf,annotator_1,annotator_1_conf,annotator_2,annotator_2_conf,annotator_3,...,annotator_9,annotator_9_conf,annotator_10,annotator_10_conf,annotator_11,annotator_11_conf,annotator_12,annotator_12_conf,annotator_13,annotator_13_conf
0,514,201,0,2,2,2,3,3,3,2,...,2,2,3,3,3,3,1,3,3,1
1,514,207,4,2,2,3,3,3,3,2,...,1,3,3,2,3,3,3,3,2,2
2,514,212,2,0,2,2,1,3,4,3,...,3,3,4,3,3,3,3,4,1,3
3,514,224,1,2,2,0,2,3,2,3,...,1,3,3,2,3,3,3,3,2,2
4,314,383,4,4,4,4,3,3,2,3,...,2,2,4,3,3,2,3,2,4,2


In [146]:
def combine_mean_and_uncertainty(avg_report, uncertainty_report):
    # Convert the DataFrame to an object dtype to allow storing strings
    combined_report = avg_report.astype(object)

    # Iterate through each row and column to apply the formatting
    for row in avg_report.index:
        for col in avg_report.columns:
            avg_value = avg_report.loc[row, col]
            std_value = uncertainty_report.loc[row, col]

            # Format the output as "avg ± std" with two decimals
            combined_report.loc[row, col] = f"{avg_value:.2f} ± {std_value:.2f}"

    return combined_report


def aggregate_reports(
    clf_reports, confidence=0.95, uncertainty_mode="expanded", transpose=True
):
    # Stack all reports and calculate mean and std
    stacked_reports = pd.concat(clf_reports)
    avg_report = stacked_reports.groupby(level=0).mean()
    std_report = stacked_reports.groupby(level=0).std()

    # Calculate standard error of the mean
    n_folds = len(clf_reports)
    sem_report = std_report / np.sqrt(n_folds)

    # Calculate expanded uncertainty with desired confidence interval
    # For 95% CI, use t-distribution with n-1 degrees of freedom
    t_value = stats.t.ppf((1 + confidence) / 2, n_folds - 1)
    match uncertainty_mode:
        case "expanded":
            uncertainty_report = sem_report * t_value
        case "sem":
            uncertainty_report = sem_report
        case "std":
            uncertainty_report = std_report
        case _:
            raise Exception("Uncertainty mode can be {expanded, sem, std}")

    # Create a function to format mean ± expanded uncertainty
    def format_with_uncertainty(mean_val, uncertainty_val):
        return f"{mean_val:.2f} ± {uncertainty_val:.2f}"

    # Create combined report with mean and expanded uncertainty
    combined_report = pd.DataFrame()
    for metric in avg_report.index:
        for class_name in avg_report.columns:
            mean_value = avg_report.loc[metric, class_name]
            uncertainty = uncertainty_report.loc[metric, class_name]
            combined_report.loc[metric, class_name] = format_with_uncertainty(
                mean_value, uncertainty
            )

    # Filter and return only the columns of interest
    if transpose:
        combined_report = combined_report.T
    filtered_report = combined_report.loc[
        ["precision", "recall", "f1-score", "support"]
    ]
    cols = ["Low", "High", "accuracy", "macro avg"]
    if "Neutral" in filtered_report.columns:
        cols.insert(1, "Neutral")
    return filtered_report[cols]

In [120]:
def evaluate_human_performance(df):
    reports = []
    annotator_columns = [c for c in df.columns if c.startswith("annotator") and not c.endswith("conf")]
    for annotator in annotator_columns:
        temp = df[["engagement", annotator]].dropna()
        reports.append(
            pd.DataFrame(
                classification_report(
                    temp["engagement"], temp[annotator], output_dict=True
                )
            ).T
        )

    return aggregate_reports(reports)

In [126]:
agg_df_annotations = df_annotations.replace(
    {0: "Low", 1: "Low", 2: "Neutral", 3: "High", 4: "High"}
)
bin_df_annotations = df_annotations.replace(
    {0: "Low", 1: "Low", 2: "Low", 3: "High", 4: "High"}
)
no_neutral_df_annotations = df_annotations[df_annotations.engagement != 2].replace(
    {0: "Low", 1: "Low", 2: None, 3: "High", 4: "High"}
)
evaluate_human_performance(agg_df_annotations)

,Low,High,accuracy,macro avg
precision,0.28 ± 0.16,0.38 ± 0.08,0.31 ± 0.07,0.30 ± 0.09
recall,0.21 ± 0.11,0.45 ± 0.13,0.31 ± 0.07,0.29 ± 0.07
f1-score,0.23 ± 0.11,0.40 ± 0.09,0.31 ± 0.07,0.28 ± 0.07
support,7.00 ± 0.00,8.00 ± 0.00,0.31 ± 0.07,20.00 ± 0.00


In [127]:
evaluate_human_performance(bin_df_annotations)

,Low,High,accuracy,macro avg
precision,0.59 ± 0.07,0.38 ± 0.08,0.50 ± 0.07,0.49 ± 0.07
recall,0.53 ± 0.10,0.45 ± 0.13,0.50 ± 0.07,0.49 ± 0.07
f1-score,0.55 ± 0.08,0.40 ± 0.09,0.50 ± 0.07,0.47 ± 0.07
support,12.00 ± 0.00,8.00 ± 0.00,0.50 ± 0.07,20.00 ± 0.00


In [128]:
evaluate_human_performance(no_neutral_df_annotations)

,Low,High,accuracy,macro avg
precision,0.36 ± 0.18,0.51 ± 0.09,0.46 ± 0.10,0.43 ± 0.12
recall,0.30 ± 0.16,0.60 ± 0.13,0.46 ± 0.10,0.45 ± 0.10
f1-score,0.30 ± 0.15,0.53 ± 0.09,0.46 ± 0.10,0.42 ± 0.10
support,5.00 ± 0.91,5.79 ± 0.65,0.46 ± 0.10,10.79 ± 1.26


## Flow-based Model

In [133]:
survey_stats = pd.read_csv(dataset_root/"Questionnaire/submissions.csv").merge(pd.read_csv(dataset_root/"Questionnaire/participants.csv"))
survey_stats

,submission_id,participant_id,game,difficulty,session_no,start_ts,end_ts,engagement,interest,stress,excitement,age,sex,fifa_exp,sf_exp
0,39,183,FIFA23,World Class,1,2023-10-23 11:34:04-0400,2023-10-23 11:37:29-0400,4,2,2,1,26,M,1,0
1,40,183,FIFA23,World Class,2,2023-10-23 11:38:12-0400,2023-10-23 11:39:25-0400,4,2,2,1,26,M,1,0
2,41,183,FIFA23,Semi-Pro,1,2023-10-23 11:45:34-0400,2023-10-23 11:46:31-0400,3,2,1,0,26,M,1,0
3,42,183,FIFA23,Semi-Pro,2,2023-10-23 11:46:55-0400,2023-10-23 11:47:59-0400,3,3,1,1,26,M,1,0
4,43,183,FIFA23,Legendary,1,2023-10-23 11:49:29-0400,2023-10-23 11:50:25-0400,3,2,1,1,26,M,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,974,458,Street Fighter V,(1),2,2023-11-13 15:24:16-0500,2023-11-13 15:24:59-0500,3,3,0,2,26,M,1,1
896,975,458,Street Fighter V,(1),3,2023-11-13 15:25:18-0500,2023-11-13 15:25:47-0500,3,2,0,2,26,M,1,1
897,976,458,Street Fighter V,(2),1,2023-11-13 15:26:37-0500,2023-11-13 15:27:19-0500,2,3,1,1,26,M,1,1
898,977,458,Street Fighter V,(2),2,2023-11-13 15:27:40-0500,2023-11-13 15:28:20-0500,3,3,0,1,26,M,1,1


In [134]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.preprocessing import LabelEncoder


def prepare_data(df):
    # Create experience column based on game
    df = df.assign(exp=np.where(df["game"] == "FIFA23", df["fifa_exp"], df["sf_exp"]))

    # Create difficulty encoders
    fifa_levels = [
        "Beginner",
        "Amateur",
        "Semi-Pro",
        "Professional",
        "World Class",
        "Legendary",
    ]
    sf_levels = ["(1)", "(2)", "(3)", "(4)", "(5)", "(6)", "(7)", "(8)"]

    fifa_diff_map = {level: i for i, level in enumerate(fifa_levels)}
    sf_diff_map = {level: i for i, level in enumerate(sf_levels)}

    # Encode difficulty based on game
    df["difficulty_encoded"] = df.apply(
        lambda row: (
            fifa_diff_map[row["difficulty"]]
            if row["game"] == "FIFA23"
            else sf_diff_map[row["difficulty"]]
        ),
        axis=1,
    )

    # Normalize difficulty to 0-1 range for each game
    df["difficulty_normalized"] = df.apply(
        lambda row: (
            row["difficulty_encoded"] / (len(fifa_levels) - 1)
            if row["game"] == "FIFA23"
            else row["difficulty_encoded"] / (len(sf_levels) - 1)
        ),
        axis=1,
    )

    # Prepare features and target
    X = df[["exp", "difficulty_normalized"]]
    y = df["engagement"]
    groups = df["participant_id"]

    return X, y, groups


# def create_cv_splits(X, y, groups, n_splits=5):
#     cv = StratifiedGroupKFold(n_splits=n_splits, shuffle=True, random_state=42)
#     return list(cv.split(X, y, groups))


def create_cv_splits(X, y, groups, splits_df_list):
    splits = []
    for df in splits_df_list:
        test_groups = set(df[df.Split == "test"].ParticipantId.unique())
        mask = groups.isin(test_groups)
        train_index = y[~mask].index
        test_index = y[mask].index
        splits.append((train_index, test_index))
    return splits


# Usage example:
# df = pd.read_csv('your_data.csv')
# X, y, groups = prepare_data(df)
# splits = create_cv_splits(X, y, groups)

# For each split:
# for train_idx, test_idx in splits:
#     X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
#     y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

In [135]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report


def train_and_evaluate(
    X,
    y,
    splits,
    model=RandomForestClassifier,
    model_kwargs={"class_weight": "balanced"},
):
    reports = []
    for train_idx, test_idx in splits:
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        clf = model(random_state=42, **model_kwargs)
        clf.fit(X_train, y_train)

        y_pred = clf.predict(X_test)
        # y_pred_proba = clf.predict_proba(X_test)

        # Get classification report as dict and convert to df
        report_dict = classification_report(y_test, y_pred, output_dict=True)
        report_df = pd.DataFrame(report_dict).transpose()

        reports.append(report_df)

    return reports


# Usage:
# reports = train_and_evaluate(X, y, splits)
# final_report = aggregate_reports(reports)

In [140]:
from pathlib import Path

splits_root = Path(dataset_root/"splits/")
splits_df_list = [pd.read_csv(fold / "0.csv") for fold in splits_root.glob("*/")]

In [147]:
# low/neutral/high
X, y, groups = prepare_data(survey_stats)
y_agg = y.map({0: "Low", 1: "Low", 2: "Neutral", 3: "High", 4: "High"})
splits = create_cv_splits(X, y_agg, groups, splits_df_list)
reports = train_and_evaluate(X, y_agg, splits)
final_report = aggregate_reports(reports)
final_report

,Low,Neutral,High,accuracy,macro avg
precision,0.27 ± 0.08,0.13 ± 0.09,0.77 ± 0.08,0.54 ± 0.07,0.39 ± 0.06
recall,0.57 ± 0.29,0.16 ± 0.13,0.66 ± 0.08,0.54 ± 0.07,0.46 ± 0.09
f1-score,0.34 ± 0.12,0.14 ± 0.11,0.70 ± 0.06,0.54 ± 0.07,0.40 ± 0.07
support,14.57 ± 6.96,23.29 ± 4.62,90.71 ± 16.97,0.54 ± 0.07,128.57 ± 16.04


In [149]:
# treat neutral as low
X, y, groups = prepare_data(survey_stats)
y_agg = y.map({0: "Low", 1: "Low", 2: "Low", 3: "High", 4: "High"})
splits = create_cv_splits(X, y_agg, groups, splits_df_list)
reports = train_and_evaluate(X, y_agg, splits)
final_report = aggregate_reports(reports)
final_report

,Low,High,accuracy,macro avg
precision,0.43 ± 0.15,0.77 ± 0.08,0.66 ± 0.06,0.60 ± 0.08
recall,0.45 ± 0.13,0.75 ± 0.07,0.66 ± 0.06,0.60 ± 0.07
f1-score,0.43 ± 0.12,0.76 ± 0.05,0.66 ± 0.06,0.59 ± 0.07
support,37.86 ± 10.08,90.71 ± 16.97,0.66 ± 0.06,128.57 ± 16.04


In [150]:
# # exclude neutral
# X, y, groups = prepare_data(survey_stats[survey_stats.engagement != 2])
# y_agg = y.map({0: "Low", 1: "Low", 3: "High", 4: "High"})
# splits = create_cv_splits(X, y_agg, groups, splits_df_list)
# reports = train_and_evaluate(X, y_agg, splits)
# final_report = aggregate_reports(reports)
# final_report[["Low", "High", "accuracy", "macro avg", "weighted avg"]]

In [151]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score, root_mean_squared_error


def train_and_evaluate_regression(X, y, splits):
    metrics = []
    for train_idx, test_idx in splits:
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        reg = LinearRegression()
        reg.fit(X_train, y_train)

        y_pred = reg.predict(X_test)

        r2 = r2_score(y_test, y_pred)
        rmse = root_mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)

        metrics.append({"r2": r2, "rmse": rmse, "mae": mae})

    metrics_df = pd.DataFrame(metrics)
    summary = pd.DataFrame({"mean": metrics_df.mean(), "std": metrics_df.std()})

    return summary


# Usage:
summary = train_and_evaluate_regression(X, y, splits)
summary

,mean,std
r2,0.023348,0.146784
rmse,0.968940,0.084231
mae,0.787810,0.081789
